# 🧩 Notebook 07: Creating Dashboards with pandas & Plotly

*From Insights to Impact*

## 📦 1. Setup & Data Loading

In [1]:
# Notebook import setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from scripts import utils_io

In [2]:
# Notebook setup
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scripts import utils_io

# Load cleaned dataset
superstore = utils_io.load_csv("../assets/superstore_final.csv")

# Convert to datetime
superstore["order_date"] = pd.to_datetime(superstore["order_date"])
superstore["ship_date"] = pd.to_datetime(superstore["ship_date"])

print("✅ Data loaded and ready for visualization.")
superstore.head()

✅ Data loaded and ready for visualization.


,order_id,customer_id,customer_name,segment,region,order_date,ship_date,category,sub_category,product_name,sales,quantity,discount,profit
0,ord-10000,cust-9476,mr. michael lopez,home office,central,2020-01-01,2020-01-03,furniture,bookcases,bookcases model 1,1292.63,5,0.3,22.94
1,ord-10001,cust-9162,robert liu,home office,south,2020-01-02,2020-01-04,furniture,bookcases,bookcases model 2,1947.16,2,0.3,205.52
2,ord-10002,cust-3824,nicole bowman,consumer,south,2020-01-03,2020-01-05,furniture,bookcases,bookcases model 3,1774.42,3,0.2,23.84
3,ord-10003,cust-8888,stephen flores,consumer,central,2020-01-04,2020-01-06,furniture,bookcases,bookcases model 4,591.01,8,0.0,64.85
4,ord-10004,cust-9980,stephen rodriguez,corporate,south,2020-01-05,2020-01-07,furniture,bookcases,bookcases model 5,1969.55,4,0.2,58.99


## 📈 Exploratory Visualizations in pandas

In [3]:
# Resample and smooth sales data
daily_sales = (
    superstore
    .set_index("order_date")
    .resample("D")["sales"]
    .sum()
    .rolling(window=7).mean()
    .reset_index()
)

# Interactive line plot
fig1 = px.line(
    daily_sales,
    x="order_date",
    y="sales",
    title="📈 7-Day Rolling Average of Daily Sales",
    labels={"sales": "Sales (Smoothed)", "order_date": "Date"},
    template="plotly_white"
)
fig1.update_traces(line=dict(color="royalblue"))
fig1.show()

In [4]:
# Boxplot: Profit Distribution by Category
fig2 = px.box(
    superstore,
    x="category",
    y="profit",
    color="category",
    title="📦 Profit Distribution by Product Category",
    hover_data=["sub_category", "region", "sales"],
    template="plotly_white"
)
fig2.update_layout(showlegend=False)
fig2.show()

In [5]:
# Histogram of Order Quantity
fig3 = px.histogram(
    superstore,
    x="quantity",
    nbins=20,
    title="📊 Distribution of Order Quantity",
    color_discrete_sequence=["indianred"],
    template="plotly_white"
)
fig3.update_layout(xaxis_title="Order Quantity", yaxis_title="Count")
fig3.show()


## 📊 3. Plotly Express Charts

In [6]:
# Bar chart: Sales by Region
# Bar chart with improved styling
region_sales = (
    superstore.groupby("region")["sales"]
    .sum()
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

fig1 = px.bar(
    region_sales,
    x="region",
    y="sales",
    title="🏙️ Total Sales by Region",
    color="region",
    text_auto=".2s",
    template="plotly_white"
)

fig1.update_layout(showlegend=False, xaxis_title="Region", yaxis_title="Sales ($)")
fig1.show()

In [7]:
# Pie Chart: Customer Segment Share
segment_share = superstore["segment"].value_counts().reset_index()
segment_share.columns = ["segment", "count"]

fig2 = px.pie(
    segment_share,
    names="segment",
    values="count",
    title="🧑‍🤝‍🧑 Customer Segment Share",
    hole=0.4,  # Donut-style
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.show()


In [8]:
# Line Chart: Monthly Profit Trend
superstore["month"] = superstore["order_date"].dt.to_period("M").astype(str)
monthly_profit = superstore.groupby("month")["profit"].sum().reset_index()

fig3 = px.line(
    monthly_profit,
    x="month",
    y="profit",
    title="📈 Monthly Profit Trend",
    markers=True,
    template="plotly_white"
)

fig3.update_layout(
    xaxis_title="Month",
    yaxis_title="Profit ($)",
    xaxis_tickangle=-45
)
fig3.update_traces(line=dict(color="green"), marker=dict(size=6))
fig3.show()


## 🔄 4. Interactive Filtering with Plotly

In [9]:
# Interactive line chart with dropdown: Region-wise Monthly Sales

# Ensure 'month' is in correct datetime order
superstore["month"] = superstore["order_date"].dt.to_period("M").astype(str)
monthly_region = (
    superstore
    .groupby(["region", "month"])["sales"]
    .sum()
    .reset_index()
    .sort_values("month")
)

# Line chart with dropdown to select region
fig = px.line(
    monthly_region,
    x="month",
    y="sales",
    color="region",
    title="🌍 Monthly Sales by Region",
    markers=True,
    template="plotly_white",
    labels={"sales": "Sales ($)", "month": "Month"},
    hover_data=["region"]
)

# Update layout for dropdown-based filtering
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Sales ($)",
    xaxis_tickangle=-45,
    updatemenus=[  # Dropdown to select regions
        {
            "buttons": [
                {"method": "update",
                 "label": region,
                 "args": [
                     {"visible": [r == region for r in monthly_region["region"].unique()]},
                     {"title": f"🌍 Monthly Sales – {region}"}
                 ]
                }
                for region in monthly_region["region"].unique()
            ] + [
                {"method": "update",
                 "label": "All",
                 "args": [
                     {"visible": [True] * len(monthly_region["region"].unique())},
                     {"title": "🌍 Monthly Sales by Region"}
                 ]
                }
            ],
            "direction": "down",
            "showactive": True,
            "x": 1.15,
            "xanchor": "right",
            "y": 1.1,
            "yanchor": "top"
        }
    ]
)

fig.show()

## 📉 5. Comparison Dashboards (Subplots)

In [10]:
# Subplots for top 3 categories by profit

# Ensure 'month' column exists and is sorted properly
superstore["month"] = superstore["order_date"].dt.to_period("M").astype(str)
superstore_sorted = superstore.sort_values("month")

# Top 3 categories by total profit
top_categories = (
    superstore_sorted.groupby("category")["profit"]
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index.tolist()
)

# Create subplot layout
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f"📦 {cat}" for cat in top_categories],
    shared_yaxes=True,
    horizontal_spacing=0.08
)

# Add one bar chart per category
for i, cat in enumerate(top_categories, 1):
    df_cat = superstore_sorted[superstore_sorted["category"] == cat]
    monthly = (
        df_cat
        .groupby("month")["profit"]
        .sum()
        .reset_index()
        .sort_values("month")
    )
    fig.add_trace(
        go.Bar(
            x=monthly["month"],
            y=monthly["profit"],
            name=cat,
            marker_color=px.colors.qualitative.Plotly[i],  # Optional: color per category
            hovertemplate="Month: %{x}<br>Profit: $%{y:.2f}<extra></extra>"
        ),
        row=1, col=i
    )

# Final layout tweaks
fig.update_layout(
    title_text="📊 Monthly Profit Trend – Top 3 Product Categories",
    template="plotly_white",
    height=400,
    showlegend=False,
    xaxis_title="Month",
    yaxis_title="Profit ($)",
    margin=dict(t=60, l=50, r=50, b=40)
)

# Rotate x-ticks for all subplots
for i in range(1, 4):
    fig.update_xaxes(tickangle=-45, row=1, col=i)

fig.show()


## 📊 6. Visual Analytics — Insights & Anomalies

In [11]:
# Scatter Plot – Sales Outliers by Category

fig = px.scatter(
    superstore,
    x="order_date",
    y="sales",
    color="category",
    size=superstore["profit"].abs(),  # size = absolute profit
    title="🚨 Sales Outliers by Product Category",
    hover_data={
        "order_date": True,
        "sales": ":.2f",
        "profit": ":.2f",
        "sub_category": True,
        "region": True
    },
    template="plotly_white",
)

fig.update_traces(marker=dict(opacity=0.6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    xaxis_title="Order Date",
    yaxis_title="Sales ($)",
    legend_title="Category",
    height=500
)
fig.show()

In [12]:
# Line Plot – 7-Day Rolling Average of Daily Sales

daily_sales = (
    superstore
    .set_index("order_date")
    .resample("D")["sales"]
    .sum()
    .rolling(window=7).mean()
    .reset_index()
)

fig2 = px.line(
    daily_sales,
    x="order_date",
    y="sales",
    title="📈 7-Day Rolling Average of Daily Sales",
    template="plotly_white",
    labels={"order_date": "Date", "sales": "Sales (Smoothed $)"}
)

fig2.update_traces(line=dict(color="mediumseagreen"), hovertemplate="Date: %{x}<br>Sales: $%{y:.2f}")
fig2.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales ($)",
    height=400
)
fig2.show()


## ✅ 7. Summary Dashboard Showcase

In [13]:
# One-shot dashboard with 2 plots side-by-side

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Prepare data
seg_sales = (
    superstore.groupby("segment")["sales"]
    .sum()
    .reset_index()
    .sort_values("sales", ascending=False)
)

reg_profit = (
    superstore.groupby("region")["profit"]
    .sum()
    .reset_index()
    .sort_values("profit", ascending=False)
)

# Create subplot layout
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("💼 Sales by Customer Segment", "🌍 Profit by Region"),
    horizontal_spacing=0.12
)

# Bar 1: Segment Sales
fig.add_trace(
    go.Bar(
        x=seg_sales["segment"],
        y=seg_sales["sales"],
        name="Segment Sales",
        marker_color="cornflowerblue",
        hovertemplate="Segment: %{x}<br>Sales: $%{y:,.2f}<extra></extra>"
    ),
    row=1, col=1
)

# Bar 2: Region Profit
fig.add_trace(
    go.Bar(
        x=reg_profit["region"],
        y=reg_profit["profit"],
        name="Region Profit",
        marker_color="darkorange",
        hovertemplate="Region: %{x}<br>Profit: $%{y:,.2f}<extra></extra>"
    ),
    row=1, col=2
)

# Layout adjustments
fig.update_layout(
    title_text="📊 Executive Overview Dashboard",
    height=500,
    template="plotly_white",
    showlegend=False,
    margin=dict(t=70, b=40, l=50, r=50),
)

fig.update_xaxes(title_text="Segment", row=1, col=1)
fig.update_xaxes(title_text="Region", row=1, col=2)
fig.update_yaxes(title_text="Sales ($)", row=1, col=1)
fig.update_yaxes(title_text="Profit ($)", row=1, col=2)

fig.show()

## 🔍 Summary

### 1. **Sales Outliers by Category (Scatter Plot)**
- Interactive scatter plot with **absolute profit as size**, and color-coded by `category`.
- Highlights extreme sales values and visually surfaces outliers.
- Hover includes `sub-category`, `region`, and actual `profit`.

### 2. **7-Day Rolling Average of Daily Sales (Line Chart)**
- Shows smoothed sales trend over time using rolling average.
- Helpful for detecting sales seasonality and weekly demand cycles.

### 3. **Profit Distribution by Category (Boxplot)**
- Interactive boxplot displaying spread and outliers in `profit` for each `category`.
- Useful for identifying risk and variability in different product lines.

### 4. **Order Quantity Distribution (Histogram)**
- Histogram visualizing frequency of order sizes.
- Can guide inventory planning and sales strategy.

### 5. **Sales by Region (Bar Chart)**
- Sorted bar chart showing total `sales` by `region`.
- Interactive and color-coded for quick regional comparison.

### 6. **Customer Segment Share (Pie Chart)**
- Donut-style pie chart showing percentage contribution of each customer segment.
- Uses color themes and inside labels for clarity.

### 7. **Monthly Profit Trend (Line Chart)**
- Line chart displaying profit movement month-by-month.
- Useful for strategic financial forecasting.

### 8. **Monthly Sales by Region (Line Chart + Dropdown)**
- Trend lines for each `region`, comparing monthly sales.
- Dropdown menu to isolate and compare individual regions.

### 9. **Top 3 Categories – Monthly Profit Trend (Subplots)**
- Multi-panel bar charts showing month-wise `profit` trends for top 3 categories.
- Shared Y-axis enables direct comparison.

### 10. **Executive Overview Dashboard (Side-by-Side Subplots)**
- Combined dashboard with:
  - Left: Sales by Segment
  - Right: Profit by Region
- Designed for leadership snapshots with minimal noise.

---

## ✅ Key Techniques Used
- `Plotly Express` and `Graph Objects` for advanced interactivity.
- `make_subplots` for side-by-side layouts.
- Rolling averages and resampling for time series smoothing.
- Use of hover templates, color themes, and layout customization for a publication-ready dashboard experience.

---

> 📁 This notebook serves as a comprehensive and modular **reference for dashboard design** using Plotly in Python.